In [ ]:
import numpy as np
import cv2,sklearn
import libsvm.svmutil as svm
from libsvm.svmutil import svm_problem,svm_parameter,svm_train,svm_predict
from sklearn.metrics import accuracy_score,confusion_matrix as cm
from sklearn.model_selection import train_test_split
import os,cvxopt
from cvxopt import matrix, solvers
from collections import Counter 

In [ ]:
def gaussian(X, gamma):
    squared_distances = np.sum(X**2, axis=1).reshape(-1, 1) + np.sum(X**2, axis=1) - 2 * np.dot(X, X.T)
    K = np.exp(-gamma * squared_distances)
    return K
def Kernel(X,gamma):
    if gamma:
        return gaussian(X, gamma)
    else:
        return np.dot(X, X.T)
def find_P(X, Y, gamma):
    K = Kernel(X, gamma)
    P=np.outer(Y,Y)*K
    return P

In [ ]:
def kernel(x_i,x_j):
    return np.exp(-0.001*(np.linalg.norm(x_i-x_j)**2))
def wtx(alpha, Y, X, x_j):
    return np.sum(alpha * Y * kernel(X, x_j))

def predict(alpha,x_train,y_train,b,x_test):
    y_pred=np.zeros(len(x_test))
    for i in range(len(y_pred)):
        y_pred[i]=wtx(alpha,y_train,x_train,x_test[i])+b
    return np.sign(y_pred)

In [ ]:
def gaussian_kernel(x1, x2, gamma):
    return np.exp(-gamma * np.linalg.norm(x1 - x2, axis=1)**2)

gamma = 0.001
C = 1
def predict2(alpha,b,x_train, y_train, x_test, gamma, C=1.0):
    decision_function = np.sum(np.outer(alpha * y_train, gaussian_kernel(x_train, x_test, gamma)), axis=0) + b

    return np.sign(decision_function)

In [ ]:
kC2classes = [['1','2'],['1','3'],['1','4'],['1','5'],['1','6'],
              ['2','3'],['2','4'],['2','5'],['2','6'],['3','4'],
              ['3','5'],['3','6'],['4','5'],['4','6'],['5','6']]

In [ ]:
x_test = []; y_test = []
classes = ['0','1','2','3','4','5']
dataset_path = 'svm/val/'
for x in classes:
    path = os.path.join(dataset_path, x)
    for y in os.listdir(path):
        image = cv2.imread(os.path.join(path, y))
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (16,16))
        image = image.reshape(768)
        image=image/255.0
        x_test.append(image)
        y_test.append(int(x)+1)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
x_test.shape,y_test.shape

In [ ]:
import random
x = [random.randint(1, 6) for p in range(0, 1200)]
accuracy_score(y_test, x)

In [ ]:
def predict_kC2(alphas, x_train, y_train, x_test, y_test):
    num_classes = 6
    y_pred = np.zeros((len(x_test), num_classes), dtype=int)
    gamma = 0.001
    
    for i, y in enumerate(alphas):
        alpha,b = y
        indices =np.where((alpha > 1e-5) & (alpha < C ))[0]
        for j in range(len(x_test)):
            pred = np.sign(np.sum(alpha * y_train*gaussian_kernel(x_train, x_test[j], gamma)) + b)
            if pred > 0:
                y_pred[j, int(kC2classes[i][1]) - 1] += 1  
            else:
                y_pred[j, int(kC2classes[i][0]) - 1] += 1
    
    y_pred = np.argmax(y_pred, axis=1) + 1
    print("Predicted labels:", y_pred)
    
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    
    return y_pred


In [ ]:
classes = ['0','1','2','3','4','5']
datas = [];labelss = []
dataset_path = 'svm/train/'
for x in classes:
    data=[];labels=[]
    path = os.path.join(dataset_path, x)
    for y in os.listdir(path):
        image = cv2.imread(os.path.join(path, y))
        image = cv2.resize(image, (16,16))
        image = image.reshape(768)
        image=image/255.0
        data.append(image)
        labels.append(int(x)+1)
    datas.append(data)
    labelss.append(labels)

In [ ]:
Abs = []
m=len(datas[0])*2
q = -np.ones((m, 1))
q = matrix(q)
B = matrix(0.0, tc='d')
G = np.vstack((-np.eye(m), np.eye(m)))
G = matrix(G)
gamma = 0.001
C=1.0
h = np.zeros((2 * m, 1))
h[m:, :] = C   
h = matrix(h)
from time import time

for X in kC2classes:
    start = time()
    data = datas[int(X[0])-1]+datas[int(X[1])-1]
    labels = [-1]*len(datas[int(X[0])-1])+[1]*len(datas[int(X[1])-1])
    
    x_train = np.array(data)
    y_train = np.array(labels)
    m = x_train.shape[0]
    P = find_P(x_train,y_train, 0.001)
    P = matrix(P)
    A = matrix(y_train.reshape(1, -1), tc='d')

    sol = solvers.qp(P, q,G, h, A, B,options={'show_progress': False}) 
    alpha = np.reshape(np.array(sol['x']), (x_train.shape[0],1))
    b = 0.0
    indices = np.where((alpha > 1e-5) & (alpha < C ))[0]
    for i in indices:
        b += labels[i] - wtx(alpha,y_train,x_train,x_train[i],indices)
    b/=len(indices)
    Abs.append([alpha,b])
    print("done",X,len(Abs),time()-start)

# pred = predict(Abs,x_train,y_train,x_test,y_test)    

In [ ]:
x_train = []; y_train = []
for i in range(6):
    x_train.extend(datas[i])
    y_train.extend(labelss[i])
x_train = np.array(x_train)
y_train = np.array(y_train)
# pred=predict(Wbs,x_train,y_train)
# cm(y_train,pred)

In [ ]:
pred  =predict_kC2(Abs, x_train, y_train, x_test, y_test)

In [ ]:
Counter(pred)

In [ ]:
# pred_t=predict(Wbs,x_test,y_test)    
cm(y_train,pred)

In [ ]:
from libsvm.svmutil import svm_problem,svm_parameter,svm_train,svm_predict
m = svm_train(y_train,x_train,'-c 10 -q -g 0.001')

In [ ]:
train = svm_predict(y_train,x_train,m)

In [ ]:
test = svm_predict(y_test,x_test,m)

In [ ]:
mis_class = []
i=0
while (i<len(test[0])) and len(mis_class)<12:
    if test[0][i] != y_test[i]:
        mis_class.append(i)
    i+=1
mis_class

In [ ]:
for i in range(12):
    mis_class[i] = ( mis_class[i],x_test[mis_class[i]])
# mis_class = np.array(mis_class)
mis_class

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
for i in range(12):
    plt.subplot(3, 4, i + 1)
    plt.imshow(mis_class[i][1].reshape(16, 16, 3))
    plt.title("Predicted: {}, but {}".format(int(test[0][i]), y_test[i]))
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
cm(y_test,test[0])

In [ ]:
cm(train[0],y_train)

In [ ]:
x_test.shape,y_test.shape

In [ ]:
test = (svm_predict(y_test,x_test,m)[0])

In [ ]:
cm(test,y_test)

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score

clf = svm.SVC(kernel='rbf', gamma=0.001, C=1.0)

clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Validation Set Accuracy:", accuracy)


In [ ]:
from collections import Counter
from sklearn.metrics import confusion_matrix
print(Counter(y_pred),Counter(y_test))
confusion_matrix(y_test,y_pred)

In [ ]:
from libsvm.svmutil import svm_problem,svm_parameter,svm_train,svm_predict
def svm_libsvm(X_train,Y_train,gamma=0.001,C=1.0):
    param = "-t 2 -c {} -q ".format(C)
    if gamma:
        param += f"-g {gamma}"
    model = svm_train(Y_train ,X_train, param)
    return model


def accuracy_util_libsvm(X_train: np.ndarray, Y_train: np.ndarray, X_test: np.ndarray, Y_test: np.ndarray, model):
    train_accuracy = svm_predict(Y_train, X_train, model, '-q')[1][0]
    test_accuracy = svm_predict(Y_test, X_test, model, '-q')[1][0]
    return train_accuracy, test_accuracy



In [ ]:
model= svm_libsvm(x_train, y_train)

In [ ]:
accuracy_util_libsvm(x_train, y_train, x_test, y_test, model)

In [ ]:
g  = 0.001
for c in {1e-5,1e-3,1,5,10}:
    print("C = {}".format(c))
    model = svm_libsvm(x_train, y_train, C=c)
    print("model done")
    train_accuracy, test_accuracy = accuracy_util_libsvm(x_train, y_train, x_test, y_test, model)
    print("C = {}, Train Accuracy = {}, Test Accuracy = {}".format(C, train_accuracy, test_accuracy))

In [ ]:
def k_fold(X,Y,c,k=5):
    m = X.shape[0]
    m //= k
    X_test, X_train = np.split(X, [m])
    Y_test, Y_train = np.split(Y, [m])
    accuracy = 0
    for i in range(k):
        clf = svm.SVC(kernel='rbf', gamma=0.001, C=c)
        clf.fit(X_train,Y_train)
        pred = clf.predict(X_test)
        accuracy += np.mean(pred == Y_test)
        if i < k - 1:
            X_train[i * m:(i + 1) * m], X_test = X_test, X_train[i * m:(i + 1) * m].copy()
            Y_train[i * m:(i + 1) * m], Y_test = Y_test, Y_train[i * m:(i + 1) * m].copy()
    return accuracy / k

In [ ]:
indices = np.arange(len(x_train))
np.random.shuffle(indices)
x_train = x_train[indices]
y_train = y_train[indices]

In [ ]:
k_fold(x_train,y_train,1)

In [ ]:
for c in [1e-5,1e-3,1,5,10]:
    print(k_fold(x_train,y_train,c))
    print('\n')
